In [287]:
# 'df' representa o DataFrame no Power BI
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder


In [288]:
df = pd.read_excel("salesfull.xlsx")



In [289]:
df['Billing USD'] = df['Billing USD'].str.replace('$', '')

In [290]:
df['Billing USD'] = df['Billing USD'].str.replace('.', '')

In [291]:
df['Billing USD'].head()

0    37520
1     3220
2    71190
3      469
4    26999
Name: Billing USD, dtype: object

In [292]:
df['Billing USD'] = pd.to_numeric(df['Billing USD'], errors='coerce')
#df['Billing USD'] = df['Billing USD'].replace('[\$,]', '', regex=True).astype(float)
#df['Billing USD'] = pd.to_numeric(df['Billing USD']).astype(float)

In [293]:
df['Billing USD'] = df['Billing USD'] / 100

In [294]:
print(df['Billing USD'].unique())

[375.2   32.2  711.9  ...  13.86 573.13 367.99]


In [295]:
df['Gender'] = df['Gender'].astype('category')
#df = pd.get_dummies(df, columns=['Gender'], prefix='Gender')

In [296]:
#df['Billing USD'] = df['Billing USD'].astype('object')

In [297]:
df.isnull().sum()

Order Number              0
Line Number               0
Order Date                0
Delivery Date             0
CustomerKey               0
StoreKey                  0
ProductKey                0
Quantity                  0
Unit Price                0
Net Price                 0
Unit Cost                 0
Currency Code             0
Exchange Rate             0
Billing Local Currency    0
Billing USD               0
Product Name              0
Manufacturer              0
Brand                     0
Category                  0
Gender                    0
City                      0
State Code                8
Country Code              0
Age                       0
Age Group                 0
TimeDelivery              0
Country                   0
dtype: int64

In [298]:
df.head()

,Order Number,Line Number,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Unit Price,Net Price,...,Brand,Category,Gender,City,State Code,Country Code,Age,Age Group,TimeDelivery,Country
0,277901,0,"quinta-feira, 10 de agosto de 2017","quinta-feira, 17 de agosto de 2017",1212508,999999,1462,1,375.20,341.43,...,Contoso,Cell phones,Male,Los Angeles,CA,US,32,young,7,United States
1,271300,0,"segunda-feira, 5 de junho de 2017","domingo, 11 de junho de 2017",1273789,999999,1346,1,32.20,28.98,...,Contoso,Cell phones,Male,San Carlos,CA,US,47,young,6,United States
2,283304,0,"terça-feira, 3 de outubro de 2017","quarta-feira, 4 de outubro de 2017",1290364,999999,2087,1,711.90,654.95,...,Contoso,Home Appliances,Male,Olney,MD,US,26,young,1,United States
3,278500,0,"quarta-feira, 16 de agosto de 2017","domingo, 20 de agosto de 2017",1324602,999999,2517,1,4.69,4.17,...,Contoso,Cell phones,Male,Norcross,GA,US,66,young,4,United States
4,269504,0,"quinta-feira, 18 de maio de 2017","sábado, 20 de maio de 2017",1337541,999999,2366,1,269.99,237.59,...,Contoso,Home Appliances,Male,Harlingen,TX,US,47,young,2,United States


In [299]:
df['Billing USD']

0         375.20
1          32.20
2         711.90
3           4.69
4         269.99
          ...   
13910    5344.81
13911      13.86
13912     123.14
13913     573.13
13914     367.99
Name: Billing USD, Length: 13915, dtype: float64

In [300]:
linhas_nulas_billing = df[df['Billing USD'].isnull()]
print(linhas_nulas_billing)

Empty DataFrame
Columns: [Order Number, Line Number, Order Date, Delivery Date, CustomerKey, StoreKey, ProductKey, Quantity, Unit Price, Net Price, Unit Cost, Currency Code, Exchange Rate, Billing Local Currency, Billing USD, Product Name, Manufacturer, Brand, Category, Gender, City, State Code, Country Code, Age, Age Group, TimeDelivery, Country]
Index: []

[0 rows x 27 columns]


In [301]:
tipos_de_dados = df.dtypes
print(tipos_de_dados)

Order Number                 int64
Line Number                  int64
Order Date                  object
Delivery Date               object
CustomerKey                  int64
StoreKey                     int64
ProductKey                   int64
Quantity                     int64
Unit Price                 float64
Net Price                  float64
Unit Cost                  float64
Currency Code               object
Exchange Rate               object
Billing Local Currency      object
Billing USD                float64
Product Name                object
Manufacturer                object
Brand                       object
Category                    object
Gender                    category
City                        object
State Code                  object
Country Code                object
Age                          int64
Age Group                   object
TimeDelivery                 int64
Country                     object
dtype: object


In [302]:
df.columns

Index(['Order Number', 'Line Number', 'Order Date', 'Delivery Date',
       'CustomerKey', 'StoreKey', 'ProductKey', 'Quantity', 'Unit Price',
       'Net Price', 'Unit Cost', 'Currency Code', 'Exchange Rate',
       'Billing Local Currency', 'Billing USD', 'Product Name', 'Manufacturer',
       'Brand', 'Category', 'Gender', 'City', 'State Code', 'Country Code',
       'Age', 'Age Group', 'TimeDelivery', 'Country'],
      dtype='object')

In [303]:
# Seleciona as variáveis relevantes
df_cluster = df[['Age', 'Billing USD', 'Category', 'Country', 'Gender', 'Product Name']]



In [304]:
# Divide as variáveis em numéricas e categóricas
numeric_features = ['Age', 'Billing USD']
categorical_features = ['Category', 'Country', 'Gender', 'Product Name']



In [305]:
# Cria transformers para aplicar a padronização nas variáveis numéricas
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])



In [306]:
# Cria transformers para aplicar a codificação one-hot nas variáveis categóricas
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])



In [307]:
# Combina os transformers em um único ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])



In [313]:
# Aplica a transformação no DataFrame
df_cluster_transformed = preprocessor.fit_transform(df_cluster)



In [314]:
# Escolhe o número de clusters
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0)
df['Cluster'] = kmeans.fit_predict(df_cluster_transformed)



In [315]:
# Visualizações de cluster usando seaborn ou matplotlib
plt.figure(figsize=(10, 6))
sns.scatterplot(x='pca_1', y='pca_2', hue='Cluster', data=df, palette='viridis', alpha=0.8)
plt.title('Clusters no Espaço PCA (2D)')
plt.show()

ValueError: Could not interpret value `pca_1` for `x`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>